<a href="https://colab.research.google.com/github/msalihk1git/Project_1/blob/main/Copy_of_Pima_Indian_Diabetes_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

: 

In [ ]:
#loading required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


: 

In [ ]:
#loading the dataset
df=pd.read_csv(r"/content/drive/MyDrive/Luminar_Projects/Project/Project_1/diabetes.csv")
df.head()

: 

**Basics** EDA


In [ ]:
#checking the unique values in Target variable
df["Outcome"].unique()

: 

In [ ]:
#plotting the target variable
sns.countplot(x="Outcome",data=df)
plt.show()

: 

In [ ]:
#checking the value count in Target variable
df["Outcome"].value_counts()

: 

In [ ]:
#Shape of the dataset
df.shape

: 

In [ ]:
#Dataset information
df.info()

: 

In [ ]:
#Identify the missing values
df.isna().sum()

: 

In [ ]:
#Describe the data
df.describe().T

: 

In [ ]:
#Correlation on Dataset based on target variable
df.corr()["Outcome"]

: 

In [ ]:
#Correlation using heat map in input Dataset

plt.figure(figsize=(6,6))
sns.heatmap(df.corr(),annot=True,cmap="YlGnBu")
plt.title("Correlation Plot - Heat Map")
plt.show()

: 

In [ ]:
#Pairplot using sns
sns.pairplot(df,hue="Outcome")

: 

In [ ]:
print("Total zero Glucose values:" + str(df[df["Glucose"]==0].shape[0]))
print("Total zero BloodPressure values:"+str(df[df["BloodPressure"]==0].shape[0]))
print("Total zero SkinThickness values:"+str(df[df["SkinThickness"]==0].shape[0]))
print("Total zero Insulin values:"+str(df[df["Insulin"]==0].shape[0]))
print("Total zero BMI' values:"+str(df[df["BMI"]==0].shape[0]))
print("Total zero DiabetesPedigreeFunction values:"+str(df[df["DiabetesPedigreeFunction"]==0].shape[0]))
print("Total zero Age values:"+str(df[df["Age"]==0].shape[0]))



: 

In [ ]:
#Missing Value treatment

df['Glucose'].loc[(df['Glucose']==0)]=df['Glucose'].median()
df['BloodPressure'].loc[(df['BloodPressure']==0)]=df['BloodPressure'].median()
df['SkinThickness'].loc[(df['SkinThickness']==0)]=df['SkinThickness'].median()
df['Insulin'].loc[(df['Insulin']==0)]=df['Insulin'].median()
df['BMI'].loc[(df['BMI']==0)]=df['BMI'].median()




: 

In [ ]:
df.head()

: 

In [ ]:
df.isna().sum()

: 

**Outlier Treatment**

In [ ]:
sns.boxplot(df["SkinThickness"])

: 

In [ ]:
df['SkinThickness'].mean()+df['SkinThickness'].std()*3

: 

In [ ]:
numericalColumns=df.select_dtypes(include=np.number).columns
numericalColumns

: 

In [ ]:
descreteCols=[]
continousCols=[]

for column in numericalColumns:
  if df[column].nunique()>20:
    continousCols.append(column)
  else:
    descreteCols.append(column)


: 

In [ ]:
continousCols

: 

In [ ]:
descreteCols

: 

In [ ]:
for column in continousCols:
  upper_lim=df[column].mean()+df[column].std()*3
  lower_lim=df[column].mean()-df[column].std()*3

  df.loc[(df[column] > upper_lim),column] = upper_lim
  df.loc[(df[column] < lower_lim),column] = lower_lim

: 

**Train Test Split**

In [ ]:
X=df.drop(["Outcome"],axis=1)
y=df["Outcome"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=10)

: 

**Scale Down**

In [ ]:
scale=StandardScaler()
X_train_scaled=scale.fit_transform(X_train)
X_test_scaled=scale.transform(X_test)

import joblib 
joblib.dump(scale,'/content/drive/MyDrive/Luminar_Projects/Project/Project_1/scale.pk1')

: 

**Apply all classification Algorithms**

In [ ]:
log =LogisticRegression()
dtc =DecisionTreeClassifier()
rf = RandomForestClassifier()
adb = AdaBoostClassifier()
grad = GradientBoostingClassifier()
svc = SVC()
knn = KNeighborsClassifier()
nbc = GaussianNB()

voting=VotingClassifier(estimators=[("Logistic_Regression",log),("Decision_Tree_Classifier",dtc),("Random_Forest_Classifier",rf),("AdaBoost_Classifier",adb),("Gragient_Boosting_Classifier",grad),("SVC",svc),("KNeighborsClassifier",knn),("GaussianNB",nbc)],voting="hard")

voting.fit(X_train,y_train)

: 

In [ ]:
for  clf in (log,dtc,rf,adb,grad,svc,knn,nbc,voting):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,clf.score(X_train,y_train))
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))
  
  print("===========================")

: 

**Hyperparameter Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV
lr_clf = LogisticRegression()
df_clf= DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
adboost_clf = AdaBoostClassifier()
grad_clf = GradientBoostingClassifier()
svc_clf = SVC()
xgb_clf = XGBClassifier()
knn_clf = KNeighborsClassifier()

clf_list=[lr_clf,df_clf,rf_clf,adboost_clf,grad_clf,svc_clf,xgb_clf,knn_clf]

grid_params_lr= [{'penalty':['l1','l2'],'solver':['saga']}]

grid_params_df =[{'criterion':["gini","entropy"], 'splitter':['best','random'],'max_depth':[3,4,5],'min_samples_split':[2,3,4],'max_features':["auto","sqrt","log2"]}]

grid_params_rf=[{'n_estimators': [4, 6, 9], 'max_features': ['log2', 'sqrt','auto'], 'criterion': ['entropy', 'gini'],'max_depth': [2, 3, 5, 10]}]

grid_params_adboost=[{'n_estimators':[10,50,250,1000],'learning_rate':[0.01,0.1],}]

grid_params_grad=[{'loss':['deviance', 'exponential'],'learning_rate':[1,7,9],'criterion':['friedman_mse','mse']}]

grid_params_svc=[{'kernel':['linear','poly','rbf'],'degree':[3,4,5]}]

grid_params_xgb=[{'booster':['gbtree', 'gblinear'],'nthread':[10,15,25]}]

grid_params_knn=[{'n_neighbors':[5,7,9,11],'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[30,50,100]}]



clf_params=[grid_params_lr,grid_params_df,grid_params_rf,grid_params_adboost,grid_params_grad,grid_params_svc,grid_params_xgb,grid_params_knn]

: 

In [ ]:
for clf,clf_param in zip(clf_list,clf_params):
  print(f"The Classifier is {clf} and its hyper params are {clf_param}")

  grid_clf = GridSearchCV(estimator=clf,param_grid=clf_param,scoring="accuracy",cv=10)
  grid_clf.fit(X_train_scaled,y_train)
  print(f"The Train accuracy for the {clf} is {grid_clf.score(X_train_scaled,y_train)}")
  print(f"The Test accuracy for the {clf} is {grid_clf.score(X_test_scaled,y_test)}")
  print(f"The best param for {clf} is {grid_clf.best_params_}")
  print("====================\n")

: 

In [ ]:
# making the best model 

rf_clf = RandomForestClassifier(criterion = 'entropy',max_depth = 3,max_features = 'log2',n_estimators = 4)
rf_clf.fit(X_train_scaled,y_train)

: 

In [ ]:
#Pickle file
import joblib 
joblib.dump(rf_clf,'/content/drive/MyDrive/Luminar_Projects/Project/Project_1/model.pkl')

: 

: 